<head>
<b><center>CIENCIA DE DATOS</center>

<center>TALLER 3</center>
<center>2024-1</center>

<center>Profesor: Gabriel Jara </center><br>
<center>Integrantes: Fernando Salgado, Cristián González</center><br>
</head>

Intro

Carga Datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


exitosas = pd.read_csv('data/transacciones_exitosas.csv',low_memory=False)
fallidas = pd.read_csv('data/transacciones_fallidas.csv')
evaluacion = pd.read_csv('data/transacciones_evaluación.csv')

# Eliminar fila con mal formato
exitosas = exitosas.drop(148541)


# Añadir etiquetas a los datos
exitosas['label'] = 0
fallidas['label'] = 1

# Combinar datos exitosos y fallidos
datos = pd.concat([exitosas, fallidas], ignore_index=True)



In [2]:
# Separar características y etiquetas
X = datos.drop(columns=['label'])
y = datos['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Fill NaN values with 0 (or use another method like column mean or median)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Optionally, check and remove or impute infinite values
# This is just a placeholder step; actual implementation may vary based on your data
X_train = X_train.replace([np.inf, -np.inf], 0)
X_test = X_test.replace([np.inf, -np.inf], 0)

# Now you can safely scale your data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# from xgboost import XGBClassifier
models = {
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(),
    'RandomForest': RandomForestClassifier(),
    # 'XGBClassifier': XGBClassifier()
}

params = {
    'LogisticRegression': {'C': [0.1, 1, 10]},
    'SVC': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'RandomForest': {'n_estimators': [100, 200], 'max_depth': [10, 20]},
    # 'XGBClassifier': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]}
}

# GridSearchCV para selección de modelos
best_models = {}
for model_name in models:
    grid = GridSearchCV(models[model_name], params[model_name], cv=5, scoring='roc_auc')
    grid.fit(X_train, y_train)
    best_models[model_name] = grid.best_estimator_
print("fin seleccion de modelo")
# Evaluar modelos
for model_name in best_models:
    y_pred = best_models[model_name].predict(X_test)
    roc = roc_auc_score(y_test, y_pred)
    print(f'{model_name}: ROC Score = {roc}')

In [5]:
# Seleccionar el mejor modelo basado en la métrica de evaluación
best_model = best_models['SVC']  # Suponiendo que XGBClassifier fue el mejor

# Entrenar el modelo con todos los datos disponibles
X_all = scaler.fit_transform(X)
best_model.fit(X_all, y)


KeyError: 'XGBClassifier'

In [ ]:
# Predecir probabilidades en el conjunto de evaluación
X_eval = scaler.transform(evaluacion)
probs = best_model.predict_proba(X_eval)[:, 1]

# Seleccionar las 200 transacciones con mayor probabilidad de ser fallidas
evaluacion['probabilidad_fallida'] = probs
top_200 = evaluacion.nlargest(200, 'probabilidad_fallida')

# Guardar los identificadores de las transacciones seleccionadas
top_200_ids = top_200.index + 1  # Asumiendo que el índice empieza en 0 y queremos ID de 1 a 1000
top_200_ids.to_csv('inspeccionar.csv', index=False, header=False)
